In [1]:
# Dependencies
### Data Cleaning
import csv
import pandas as pd
import numpy as np

### Reflecting DB
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func
# Base = declarative_base()


In [2]:
# Read Tab seperated CCES Raw data (using pd.read_csv method) into a pandas Data Frame

file = "data_raw/CCES16_Common_OUTPUT_Feb2018_VV-subset.tab"
file_df = pd.read_csv(file, sep='\t')
file_df.head(2)

,V101,lookupzip,inputstate,countyfips,birthyr,gender,educ,CC16_330a,CC16_330b,CC16_330d,CC16_330e,CC16_332a,CC16_332b,CC16_332c,CC16_332d,CC16_332e,CC16_335
0,222168628,3442,33,33011.0,1969,2,2,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0
1,273691199,70118,22,22071.0,1994,2,2,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [3]:
# Read CSV file containing State strings with matching V101 numbers

StatesCSV = 'data_raw/state_names.csv'
StatesDf = pd.read_csv(StatesCSV)
StatesDf.head(2)

,V101,StateName
0,222168628,New Hampshire
1,273691199,Louisiana


In [4]:
# Merge the two Pandas Data Frames on V101

FinalDF = pd.merge(file_df, StatesDf, on = 'V101')
FinalDF.head(2)

,V101,lookupzip,inputstate,countyfips,birthyr,gender,educ,CC16_330a,CC16_330b,CC16_330d,CC16_330e,CC16_332a,CC16_332b,CC16_332c,CC16_332d,CC16_332e,CC16_335,StateName
0,222168628,3442,33,33011.0,1969,2,2,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,New Hampshire
1,273691199,70118,22,22071.0,1994,2,2,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Louisiana


In [5]:
FinalDF.shape

(64600, 18)

In [6]:
coords = 'data_raw/statelatlong.csv'
coordsDF = pd.read_csv(coords)
coordsDF.head(2)

,StateAbb,Latitude,Longitude,StateName
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska


In [7]:
coordsDF.shape

(51, 4)

In [8]:
DF_withcoords = FinalDF.merge(coordsDF, on ='StateName')
DF_withcoords.head(2)

,V101,lookupzip,inputstate,countyfips,birthyr,gender,educ,CC16_330a,CC16_330b,CC16_330d,...,CC16_332a,CC16_332b,CC16_332c,CC16_332d,CC16_332e,CC16_335,StateName,StateAbb,Latitude,Longitude
0,222168628,3442,33,33011.0,1969,2,2,1.0,2.0,2.0,...,1.0,2.0,1.0,2.0,2.0,1.0,New Hampshire,NH,44.001231,-71.579923
1,301400441,3303,33,33013.0,1948,2,3,1.0,2.0,1.0,...,1.0,2.0,2.0,2.0,2.0,1.0,New Hampshire,NH,44.001231,-71.579923


In [9]:
DF_withcoords.shape

(64600, 21)

In [10]:
# Clean the merged Data:

# Drop all rows that have an NA in any column
file_df = DF_withcoords.dropna()

# Reset the index so there are not missing index values
file_df1 = file_df.reset_index()

# drop second index
file_df1 = file_df1.drop(columns=['index', 'inputstate'])

file_df1.head(2)

,V101,lookupzip,countyfips,birthyr,gender,educ,CC16_330a,CC16_330b,CC16_330d,CC16_330e,CC16_332a,CC16_332b,CC16_332c,CC16_332d,CC16_332e,CC16_335,StateName,StateAbb,Latitude,Longitude
0,222168628,3442,33011.0,1969,2,2,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,New Hampshire,NH,44.001231,-71.579923
1,301400441,3303,33013.0,1948,2,3,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,New Hampshire,NH,44.001231,-71.579923


In [11]:
file_df1.shape

(62881, 20)

In [12]:
# Rename columns with more descriptive titles

file_dfRename = file_df1.rename(columns = {
    'CC16_330a':'GunBackgroundChecks_16',
    'CC16_330b':'ProhibitPublication_16',
    'CC16_330d':'BanAssultWeapons_16',
    'CC16_330e':'MakeCCPEasier_16',
    'CC16_332a':'AlwaysAllowChoice_16',
    'CC16_332b':'RapeIncestorHealth_16',
    'CC16_332c':'ProhibitMoreThan20Weeks_16',
    'CC16_332d':'EmployersDeclineBenefits_16',
    'CC16_332e':'ProhibitFedFunds_16',
    'CC16_335': 'GayMarriage'
})

# Reorder the columns

CCES_df = file_dfRename[['V101', 'lookupzip', 'StateName', 'StateAbb', 'Latitude', 'Longitude', 'countyfips', 'birthyr', 'gender', 'educ',
       'GunBackgroundChecks_16', 'ProhibitPublication_16',
       'BanAssultWeapons_16', 'MakeCCPEasier_16', 'AlwaysAllowChoice_16',
       'RapeIncestorHealth_16', 'ProhibitMoreThan20Weeks_16',
       'EmployersDeclineBenefits_16', 'ProhibitFedFunds_16', 'GayMarriage']]
CCES_df.head(2)

,V101,lookupzip,StateName,StateAbb,Latitude,Longitude,countyfips,birthyr,gender,educ,GunBackgroundChecks_16,ProhibitPublication_16,BanAssultWeapons_16,MakeCCPEasier_16,AlwaysAllowChoice_16,RapeIncestorHealth_16,ProhibitMoreThan20Weeks_16,EmployersDeclineBenefits_16,ProhibitFedFunds_16,GayMarriage
0,222168628,3442,New Hampshire,NH,44.001231,-71.579923,33011.0,1969,2,2,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0
1,301400441,3303,New Hampshire,NH,44.001231,-71.579923,33013.0,1948,2,3,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0


In [13]:
########## Convert some of the columns from number answers to string value responses

# Convert gender column
CCES_df['gender'] = CCES_df['gender'].apply(lambda x: 'male' if x== 1 else 'female')

# map and convert edu column
CCES_df['educ'] = CCES_df['educ'].map({
                                            1: 'NoHS',
                                            2: 'HS_Grad',
                                            3: 'Some College',
                                            4: '2-year',
                                            5: '4-year',
                                            6: 'Post-Grad'
                                          })

# CC16_330a 1 support, 2 oppose, 8 skipped / Gun Background Checks_16
CCES_df['GunBackgroundChecks_16'] = CCES_df['GunBackgroundChecks_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_330b 1 support, 2 oppose, 8 skipped  'CC16_330b':'Prohibit Publication_16',
CCES_df['ProhibitPublication_16'] = CCES_df['ProhibitPublication_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_330d 1 support, 2 oppose, 8 skipped 'CC16_330d':'Ban Assult Weapons_16',
CCES_df['BanAssultWeapons_16'] = CCES_df['BanAssultWeapons_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_330e 1 support, 2 oppose, 8 skipped 'CC16_330e':'Make CCP Easier_16',
CCES_df['MakeCCPEasier_16'] = CCES_df['MakeCCPEasier_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332a 1 support, 2 oppose, 8 skipped 'CC16_332a':'Always Allow Choice_16',
CCES_df['AlwaysAllowChoice_16'] = CCES_df['AlwaysAllowChoice_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332b 1 support, 2 oppose, 8 skipped   'CC16_332b':'Rape, Incest, or Health_16',
CCES_df['RapeIncestorHealth_16'] = CCES_df['RapeIncestorHealth_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332c 1 support, 2 oppose, 8 skipped 'CC16_332c':'Prohibit MoreThan20 Weeks_16',
CCES_df['ProhibitMoreThan20Weeks_16'] = CCES_df['ProhibitMoreThan20Weeks_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332d 1 support, 2 oppose, 8 skipped   'CC16_332d':'Employers decline benefits_16',
CCES_df['EmployersDeclineBenefits_16'] = CCES_df['EmployersDeclineBenefits_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332e 1 support, 2 oppose, 8 skipped  'CC16_332e':'Prohibit Fed Funds_16',
CCES_df['ProhibitFedFunds_16'] = CCES_df['ProhibitFedFunds_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_335 1 support, 2 oppose, 8 skipped  'CC16_335': 'GayMarriage'
CCES_df['GayMarriage'] = CCES_df['GayMarriage'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})
CCES_df.head(2)

,V101,lookupzip,StateName,StateAbb,Latitude,Longitude,countyfips,birthyr,gender,educ,GunBackgroundChecks_16,ProhibitPublication_16,BanAssultWeapons_16,MakeCCPEasier_16,AlwaysAllowChoice_16,RapeIncestorHealth_16,ProhibitMoreThan20Weeks_16,EmployersDeclineBenefits_16,ProhibitFedFunds_16,GayMarriage
0,222168628,3442,New Hampshire,NH,44.001231,-71.579923,33011.0,1969,female,HS_Grad,Support,Oppose,Oppose,Oppose,Support,Oppose,Support,Oppose,Oppose,Support
1,301400441,3303,New Hampshire,NH,44.001231,-71.579923,33013.0,1948,female,Some College,Support,Oppose,Support,Oppose,Support,Oppose,Oppose,Oppose,Oppose,Support


In [14]:
## Troubleshooting to add up the total responses for each column
## If the responses for each section show a difference value of 0 - that's good news!

GunBcSupport = (len(CCES_df[CCES_df['GunBackgroundChecks_16']=='Support']))
GunBCOppose = (len(CCES_df[CCES_df['GunBackgroundChecks_16']=='Oppose']))
GunBcTotal = (len(CCES_df[CCES_df['GunBackgroundChecks_16']!='none']))
print('Gun Support Total Support: ', GunBcSupport)
print('Total Oppose: ', GunBCOppose)
print('Total Rows: ', GunBcTotal)
print('Total Difference', (GunBcTotal - GunBCOppose - GunBcSupport))
print('--------------------------------------------------------------------------')

ProhibitPubSupport = (len(CCES_df[CCES_df['ProhibitPublication_16']=='Support']))
ProhibitPubOppose = (len(CCES_df[CCES_df['ProhibitPublication_16']=='Oppose']))
ProhibitPubTotal = (len(CCES_df[CCES_df['ProhibitPublication_16']!='none']))
print('Prohibit Publication_16: ', ProhibitPubSupport)
print('Total Oppose: ', ProhibitPubOppose)
print('Total Rows: ', ProhibitPubTotal)
print('Total Difference', (ProhibitPubTotal - ProhibitPubOppose - ProhibitPubSupport))
print('--------------------------------------------------------------------------')
BanAssultSupport = (len(CCES_df[CCES_df['BanAssultWeapons_16']=='Support']))
BanAssultOppose = (len(CCES_df[CCES_df['BanAssultWeapons_16']=='Oppose']))
BanAssultTotal = (len(CCES_df[CCES_df['BanAssultWeapons_16']!='none']))
print('Gun Support Total Support: ', BanAssultSupport)
print('Total Oppose: ', BanAssultOppose)
print('Total Rows: ', BanAssultTotal)
print('Total Difference', (BanAssultTotal - BanAssultOppose - BanAssultSupport))
print('--------------------------------------------------------------------------')
#########################################################################################
CCPSupport = (len(CCES_df[CCES_df['MakeCCPEasier_16']=='Support']))
CCPOppose = (len(CCES_df[CCES_df['MakeCCPEasier_16']=='Oppose']))
CCPTotal = (len(CCES_df[CCES_df['MakeCCPEasier_16']!='none']))
print('Gun Support Total Support: ', CCPSupport)
print('Total Oppose: ', CCPOppose)
print('Total Rows: ', CCPTotal)
print('Total Difference', (CCPTotal - CCPOppose - CCPSupport))
print('--------------------------------------------------------------------------')

AlwaysSupport = (len(CCES_df[CCES_df['AlwaysAllowChoice_16']=='Support']))
AlwaysOppose = (len(CCES_df[CCES_df['AlwaysAllowChoice_16']=='Oppose']))
AlwaysTotal = (len(CCES_df[CCES_df['AlwaysAllowChoice_16']!='none']))
print('Prohibit Publication_16: ', AlwaysSupport)
print('Total Oppose: ', AlwaysOppose)
print('Total Rows: ', AlwaysTotal)
print('Total Difference', (AlwaysTotal - AlwaysOppose - AlwaysSupport))
print('--------------------------------------------------------------------------')
HealthSupport = (len(CCES_df[CCES_df['RapeIncestorHealth_16']=='Support']))
HealthOppose = (len(CCES_df[CCES_df['RapeIncestorHealth_16']=='Oppose']))
HealthTotal = (len(CCES_df[CCES_df['RapeIncestorHealth_16']!='none']))
print('Gun Support Total Support: ', HealthSupport)
print('Total Oppose: ', HealthOppose)
print('Total Rows: ', HealthTotal)
print('Total Difference', (HealthTotal - HealthOppose - HealthSupport))
print('--------------------------------------------------------------------------')

#########################################################################################
MoreThan20Support = (len(CCES_df[CCES_df['ProhibitMoreThan20Weeks_16']=='Support']))
MoreThan20Oppose = (len(CCES_df[CCES_df['ProhibitMoreThan20Weeks_16']=='Oppose']))
MoreThan20Total = (len(CCES_df[CCES_df['ProhibitMoreThan20Weeks_16']!='none']))
print('Gun Support Total Support: ', MoreThan20Support)
print('Total Oppose: ', MoreThan20Oppose)
print('Total Rows: ', MoreThan20Total)
print('Total Difference', (MoreThan20Total - MoreThan20Oppose - MoreThan20Support))
print('--------------------------------------------------------------------------')

DeclineBenSupport = (len(CCES_df[CCES_df['EmployersDeclineBenefits_16']=='Support']))
DeclineBenOppose = (len(CCES_df[CCES_df['EmployersDeclineBenefits_16']=='Oppose']))
DeclineBenTotal = (len(CCES_df[CCES_df['EmployersDeclineBenefits_16']!='none']))
print('Prohibit Publication_16: ', DeclineBenSupport)
print('Total Oppose: ', DeclineBenOppose)
print('Total Rows: ', DeclineBenTotal)
print('Total Difference', (DeclineBenTotal - DeclineBenOppose - DeclineBenSupport))
print('--------------------------------------------------------------------------')
NoFedFundsSupport = (len(CCES_df[CCES_df['ProhibitFedFunds_16']=='Support']))
NoFedFundsOppose = (len(CCES_df[CCES_df['ProhibitFedFunds_16']=='Oppose']))
NoFedFundsTotal = (len(CCES_df[CCES_df['ProhibitFedFunds_16']!='none']))
print('Gun Support Total Support: ', NoFedFundsSupport)
print('Total Oppose: ', NoFedFundsOppose)
print('Total Rows: ', NoFedFundsTotal)
print('Total Difference', (NoFedFundsTotal - NoFedFundsOppose - NoFedFundsSupport))
print('--------------------------------------------------------------------------')

#########################################################################################
GaySupport = (len(CCES_df[CCES_df['GayMarriage']=='Support']))
GayOppose = (len(CCES_df[CCES_df['GayMarriage']=='Oppose']))
GayTotal = (len(CCES_df[CCES_df['GayMarriage']!='none']))
print('GayMarriage Total Support: ', GaySupport)
print('Total Oppose: ', GayOppose)
print('Total Rows: ', GayTotal)
print('Total Difference', (GayTotal - GayOppose - GaySupport))
print('--------------------------------------------------------------------------')

GenderMale = (len(CCES_df[CCES_df['gender']=='male']))
GenderFemale = (len(CCES_df[CCES_df['gender']=='female']))
GenderTotal = (len(CCES_df[CCES_df['gender']!='none']))
print('gender: ', GenderMale)
print('Total Male: ', GenderMale)
print('Total Female: ', GenderFemale)
print('Total Rows: ', GenderTotal)
print('Total Difference', (GenderTotal - GenderFemale - GenderMale))
print('--------------------------------------------------------------------------')

# Education

NoHS = (len(CCES_df[CCES_df['educ']=='NoHS']))
HS_Grad = (len(CCES_df[CCES_df['educ']=='HS_Grad']))
SomeCollege = (len(CCES_df[CCES_df['educ']=='Some College']))
year2 = (len(CCES_df[CCES_df['educ']=='2-year']))
year4= (len(CCES_df[CCES_df['educ']=='4-year']))
PostGrad = (len(CCES_df[CCES_df['educ']=='Post-Grad']))
eduTotal = (len(CCES_df[CCES_df['educ']!='none']))
print('educ Total Support: ' )
print('Total NoHS: ',NoHS )
print('Total HS_Grad: ', HS_Grad)
print('Total SomeCollege: ', SomeCollege)
print('Total year2: ', year2)
print('Total year4: ', year4)
print('Total PostGrad: ', PostGrad)
print('Total Rows: ', eduTotal)
print('Total Difference', (eduTotal - NoHS - HS_Grad- SomeCollege - year2 - year4 - PostGrad ))
print('--------------------------------------------------------------------------')


Gun Support Total Support:  56810
Total Oppose:  6071
Total Rows:  62881
Total Difference 0
--------------------------------------------------------------------------
Prohibit Publication_16:  37098
Total Oppose:  25783
Total Rows:  62881
Total Difference 0
--------------------------------------------------------------------------
Gun Support Total Support:  41793
Total Oppose:  21088
Total Rows:  62881
Total Difference 0
--------------------------------------------------------------------------
Gun Support Total Support:  23663
Total Oppose:  39218
Total Rows:  62881
Total Difference 0
--------------------------------------------------------------------------
Prohibit Publication_16:  38869
Total Oppose:  24012
Total Rows:  62881
Total Difference 0
--------------------------------------------------------------------------
Gun Support Total Support:  28768
Total Oppose:  34113
Total Rows:  62881
Total Difference 0
------------------------------------------------------------------------

In [15]:
###### Option 1 using Declarative Base

Base = declarative_base()

class Cces_16(Base):
    __tablename__ = 'Cces_16'
    index = Column(Integer, primary_key=True)
    V101 = Column(Integer)
    lookupzip = Column(Integer)
    StateName = Column(String(255))
    StateAbb = Column(String(255))
    Latitude = Column(Float)
    Longitude = Column(Float)
    countyfips = Column(Float)
    birthyr = Column(Integer)
    gender = Column(String(255))
    educ = Column(String(255))
    GunBackgroundChecks_16 = Column(String(255))
    ProhibitPublication_16 = Column(String(255))
    BanAssultWeapons_16 = Column(String(255))
    MakeCCPEasier_16 = Column(String(255))
    AlwaysAllowChoice_16 = Column(String(255))
    RapeIncestorHealth_16 = Column(String(255))
    ProhibitMoreThan20Weeks_16 = Column(String(255))
    Employersdeclinebenefits_16 = Column(String(255))
    ProhibitFedFunds_16 = Column(String(255))
    GayMarriage = Column(String(255))

engine = create_engine("sqlite:///db/CCES_Ver61.sqlite")

Base.metadata.create_all(engine)

session = Session(bind=engine)

In [25]:
# Loop to transfer/write the DF data into the sqlite file.
# The only way I could get this to work was in a loop where the notebook wasn't trying to write more than 50 rows at a time.
# I believe this is a hardware limitation of my laptop where it could not process more than 50 rows at a time in the local
# memory without the notebook becoming unresponsive. This method has worked accuratly and quickly for 10+ consecutive tests.

# It takes approximately 1 minute for this cell to run.

x1 = 0
x2 = 25

while x1 < 63000:
    
    tempDF = CCES_df.iloc[x1:x2]
    tempDF.to_sql('Cces_16', con=engine, if_exists='append')
    x1 += 25
    x2 += 25
    print('Round complete, starting', x1, 'to', x2)
    
    

C:\Users\dkrei\Anaconda3\envs\PythonData\lib\site-packages\pandas\io\sql.py:1166: UserWarning: The provided table name 'CCES_16' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


Round complete, starting 25 to 50
Round complete, starting 50 to 75
Round complete, starting 75 to 100
Round complete, starting 100 to 125
Round complete, starting 125 to 150
Round complete, starting 150 to 175
Round complete, starting 175 to 200
Round complete, starting 200 to 225
Round complete, starting 225 to 250
Round complete, starting 250 to 275
Round complete, starting 275 to 300
Round complete, starting 300 to 325
Round complete, starting 325 to 350
Round complete, starting 350 to 375
Round complete, starting 375 to 400
Round complete, starting 400 to 425
Round complete, starting 425 to 450
Round complete, starting 450 to 475
Round complete, starting 475 to 500
Round complete, starting 500 to 525
Round complete, starting 525 to 550
Round complete, starting 550 to 575
Round complete, starting 575 to 600
Round complete, starting 600 to 625
Round complete, starting 625 to 650
Round complete, starting 650 to 675
Round complete, starting 675 to 700
Round complete, starting 700 to 7

Round complete, starting 5475 to 5500
Round complete, starting 5500 to 5525
Round complete, starting 5525 to 5550
Round complete, starting 5550 to 5575
Round complete, starting 5575 to 5600
Round complete, starting 5600 to 5625
Round complete, starting 5625 to 5650
Round complete, starting 5650 to 5675
Round complete, starting 5675 to 5700
Round complete, starting 5700 to 5725
Round complete, starting 5725 to 5750
Round complete, starting 5750 to 5775
Round complete, starting 5775 to 5800
Round complete, starting 5800 to 5825
Round complete, starting 5825 to 5850
Round complete, starting 5850 to 5875
Round complete, starting 5875 to 5900
Round complete, starting 5900 to 5925
Round complete, starting 5925 to 5950
Round complete, starting 5950 to 5975
Round complete, starting 5975 to 6000
Round complete, starting 6000 to 6025
Round complete, starting 6025 to 6050
Round complete, starting 6050 to 6075
Round complete, starting 6075 to 6100
Round complete, starting 6100 to 6125
Round comple

Round complete, starting 10875 to 10900
Round complete, starting 10900 to 10925
Round complete, starting 10925 to 10950
Round complete, starting 10950 to 10975
Round complete, starting 10975 to 11000
Round complete, starting 11000 to 11025
Round complete, starting 11025 to 11050
Round complete, starting 11050 to 11075
Round complete, starting 11075 to 11100
Round complete, starting 11100 to 11125
Round complete, starting 11125 to 11150
Round complete, starting 11150 to 11175
Round complete, starting 11175 to 11200
Round complete, starting 11200 to 11225
Round complete, starting 11225 to 11250
Round complete, starting 11250 to 11275
Round complete, starting 11275 to 11300
Round complete, starting 11300 to 11325
Round complete, starting 11325 to 11350
Round complete, starting 11350 to 11375
Round complete, starting 11375 to 11400
Round complete, starting 11400 to 11425
Round complete, starting 11425 to 11450
Round complete, starting 11450 to 11475
Round complete, starting 11475 to 11500


Round complete, starting 16125 to 16150
Round complete, starting 16150 to 16175
Round complete, starting 16175 to 16200
Round complete, starting 16200 to 16225
Round complete, starting 16225 to 16250
Round complete, starting 16250 to 16275
Round complete, starting 16275 to 16300
Round complete, starting 16300 to 16325
Round complete, starting 16325 to 16350
Round complete, starting 16350 to 16375
Round complete, starting 16375 to 16400
Round complete, starting 16400 to 16425
Round complete, starting 16425 to 16450
Round complete, starting 16450 to 16475
Round complete, starting 16475 to 16500
Round complete, starting 16500 to 16525
Round complete, starting 16525 to 16550
Round complete, starting 16550 to 16575
Round complete, starting 16575 to 16600
Round complete, starting 16600 to 16625
Round complete, starting 16625 to 16650
Round complete, starting 16650 to 16675
Round complete, starting 16675 to 16700
Round complete, starting 16700 to 16725
Round complete, starting 16725 to 16750


...

Round complete, starting 36975 to 37000
Round complete, starting 37000 to 37025
Round complete, starting 37025 to 37050
Round complete, starting 37050 to 37075
Round complete, starting 37075 to 37100
Round complete, starting 37100 to 37125
Round complete, starting 37125 to 37150
Round complete, starting 37150 to 37175
Round complete, starting 37175 to 37200
Round complete, starting 37200 to 37225
Round complete, starting 37225 to 37250
Round complete, starting 37250 to 37275
Round complete, starting 37275 to 37300
Round complete, starting 37300 to 37325
Round complete, starting 37325 to 37350
Round complete, starting 37350 to 37375
Round complete, starting 37375 to 37400
Round complete, starting 37400 to 37425
Round complete, starting 37425 to 37450
Round complete, starting 37450 to 37475
Round complete, starting 37475 to 37500
Round complete, starting 37500 to 37525
Round complete, starting 37525 to 37550
Round complete, starting 37550 to 37575
Round complete, starting 37575 to 37600


Round complete, starting 42150 to 42175
Round complete, starting 42175 to 42200
Round complete, starting 42200 to 42225
Round complete, starting 42225 to 42250
Round complete, starting 42250 to 42275
Round complete, starting 42275 to 42300
Round complete, starting 42300 to 42325
Round complete, starting 42325 to 42350
Round complete, starting 42350 to 42375
Round complete, starting 42375 to 42400
Round complete, starting 42400 to 42425
Round complete, starting 42425 to 42450
Round complete, starting 42450 to 42475
Round complete, starting 42475 to 42500
Round complete, starting 42500 to 42525
Round complete, starting 42525 to 42550
Round complete, starting 42550 to 42575
Round complete, starting 42575 to 42600
Round complete, starting 42600 to 42625
Round complete, starting 42625 to 42650
Round complete, starting 42650 to 42675
Round complete, starting 42675 to 42700
Round complete, starting 42700 to 42725
Round complete, starting 42725 to 42750
Round complete, starting 42750 to 42775


Round complete, starting 47325 to 47350
Round complete, starting 47350 to 47375
Round complete, starting 47375 to 47400
Round complete, starting 47400 to 47425
Round complete, starting 47425 to 47450
Round complete, starting 47450 to 47475
Round complete, starting 47475 to 47500
Round complete, starting 47500 to 47525
Round complete, starting 47525 to 47550
Round complete, starting 47550 to 47575
Round complete, starting 47575 to 47600
Round complete, starting 47600 to 47625
Round complete, starting 47625 to 47650
Round complete, starting 47650 to 47675
Round complete, starting 47675 to 47700
Round complete, starting 47700 to 47725
Round complete, starting 47725 to 47750
Round complete, starting 47750 to 47775
Round complete, starting 47775 to 47800
Round complete, starting 47800 to 47825
Round complete, starting 47825 to 47850
Round complete, starting 47850 to 47875
Round complete, starting 47875 to 47900
Round complete, starting 47900 to 47925
Round complete, starting 47925 to 47950


Round complete, starting 52575 to 52600
Round complete, starting 52600 to 52625
Round complete, starting 52625 to 52650
Round complete, starting 52650 to 52675
Round complete, starting 52675 to 52700
Round complete, starting 52700 to 52725
Round complete, starting 52725 to 52750
Round complete, starting 52750 to 52775
Round complete, starting 52775 to 52800
Round complete, starting 52800 to 52825
Round complete, starting 52825 to 52850
Round complete, starting 52850 to 52875
Round complete, starting 52875 to 52900
Round complete, starting 52900 to 52925
Round complete, starting 52925 to 52950
Round complete, starting 52950 to 52975
Round complete, starting 52975 to 53000
Round complete, starting 53000 to 53025
Round complete, starting 53025 to 53050
Round complete, starting 53050 to 53075
Round complete, starting 53075 to 53100
Round complete, starting 53100 to 53125
Round complete, starting 53125 to 53150
Round complete, starting 53150 to 53175
Round complete, starting 53175 to 53200


Round complete, starting 57825 to 57850
Round complete, starting 57850 to 57875
Round complete, starting 57875 to 57900
Round complete, starting 57900 to 57925
Round complete, starting 57925 to 57950
Round complete, starting 57950 to 57975
Round complete, starting 57975 to 58000
Round complete, starting 58000 to 58025
Round complete, starting 58025 to 58050
Round complete, starting 58050 to 58075
Round complete, starting 58075 to 58100
Round complete, starting 58100 to 58125
Round complete, starting 58125 to 58150
Round complete, starting 58150 to 58175
Round complete, starting 58175 to 58200
Round complete, starting 58200 to 58225
Round complete, starting 58225 to 58250
Round complete, starting 58250 to 58275
Round complete, starting 58275 to 58300
Round complete, starting 58300 to 58325
Round complete, starting 58325 to 58350
Round complete, starting 58350 to 58375
Round complete, starting 58375 to 58400
Round complete, starting 58400 to 58425
Round complete, starting 58425 to 58450


In [27]:
# Declare a base using automap_base()
Base = automap_base()
Base

sqlalchemy.ext.automap.Base

In [28]:
# Use the base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [29]:
# Print all of the classses mapped to the Base (this provides a list of the table names in the sqlite.db)
Base.classes.keys()

['Cces_16']

In [34]:
# Assign the CCES_16 class to a variable called 'CCES'
CCES = Base.classes.Cces_16

In [37]:

sample = GayOppose
preFix = "Cces."
newSample = globals()[preFix+sample]
print(newSample)

TypeError: must be str, not int

In [35]:
### Accuracy Test 1
# Query support for gay marriage from pandas DF and from Sqlite file

ApproveGay = session.query(CCES).filter(CCES.GayMarriage == 'Support').count()
print(f'Sqlite query of votes supporting Gay Marrige: {ApproveGay}')
print(f'Pandas DF Sort of votes supporing Gay Marriage: {GaySupport}')

Sqlite query of votes supporting Gay Marrige: 40955
Pandas DF Sort of votes supporing Gay Marriage: 40955


In [36]:
### Accuracy Test 2
# Query support for Gun Background Checks from pandas DF and Sqlite file
SupportChecks = session.query(CCES).filter(CCES.GunBackgroundChecks_16 == 'Support').count()
print(f'Sqlite support votes: {SupportChecks}')
print(f'Pandas DF support votes: {GunBcSupport}')

Sqlite support votes: 56810
Pandas DF support votes: 56810


In [ ]:
### Multi Filter Test 1
# Query Males who support background checks
# Query Females who support background checks
# Total number of support background check votes in Pandas

MaleCheckSupport = session.query(CCES).filter(CCES.gender == 'male').filter(CCES.GunBackgroundChecks_16 == 'Support').count()
FemaleCheckSupport = session.query(CCES).filter(CCES.gender == 'female').filter(CCES.GunBackgroundChecks_16 == 'Support').count()

print(f'Query Male = {MaleCheckSupport}')
print(f'Query Female = {FemaleCheckSupport}')
print('Query total = ', MaleCheckSupport + FemaleCheckSupport)
print(f'Total Support = {GunBcSupport}')

In [ ]:
# Filter by state how many respondents were supportive of Gun Background Checks

StatesFor = session.query(CCES.StateName, func.count(CCES.GunBackgroundChecks_16)).group_by(CCES.StateName).all()
StatesFor[0:5]